## <b><font color='darkblue'>Preface</font></b>
([source](https://learn.deeplearning.ai/courses/langchain/lesson/2/models%2C-prompts-and-parsers)) <b><font size='3ptx'>From this chapter, we are going to see how to use OpenAI APIs and how the LangChain package integrates OpenAI.</font></b>

<a id='sect0'></a>
### <b><font color='darkgreen'>Outline</font></b>
* <b><font size='3ptx'><a href='#sect1'>Direct API calls to OpenAI</a></font></b>
* <b><font size='3ptx'><a href='#sect2'>API calls through LangChain:</a></font></b>
    * [Prompts](#sect2_1)
    * [Models](#sect2_2)
    * [Output parsers](#sect2_3)

### <b><font color='darkgreen'>Get your [OpenAI API Key](https://platform.openai.com/account/api-keys)</font></b>
First thing first, we need to get the OpenAPI key in order to use the APIs they provided.

In [1]:
#!pip install python-dotenv
#!pip install openai

In [20]:
import datetime
import os
import openai
from dotenv import load_dotenv, find_dotenv
from openai import OpenAI

a = load_dotenv(find_dotenv(os.path.expanduser('~/.env'))) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']
client = OpenAI()

<b><font color='darkred'>Note</font></b>: LLM's do not always produce the same results. When executing the code in your notebook, you may get slightly different answers that those in [the video](https://learn.deeplearning.ai/courses/langchain/lesson/2/models%2C-prompts-and-parsers).

In [11]:
# Get the current date
current_date = datetime.datetime.now().date()

# Define the date after which the model should be set to "gpt-3.5-turbo"
target_date = datetime.date(2024, 6, 12)

# Set the model variable based on the current date
if current_date > target_date:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0301"

In [14]:
llm_model

'gpt-3.5-turbo'

<a id='sect1'></a>
## <b><font color='darkblue'>Chat API : OpenAI</font></b>
Let's start with a direct API calls to OpenAI.

In [23]:
def get_completion(prompt, model=llm_model):
    
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}]
    )    
    return response.choices[0].message.content.strip()

In [24]:
get_completion("What is 1+1?")

'The sum of 1+1 is 2.'

### <b><font color='darkgreen'>Email format/style example</font></b>

In [27]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse,\
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

style = """American English \
in a calm and respectful tone
"""

prompt = f"""Translate the text \
that is delimited by triple backticks 
into a style that is {style}.
text: ```{customer_email}```
"""

print(prompt)

Translate the text that is delimited by triple backticks 
into a style that is American English in a calm and respectful tone
.
text: ```
Arrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse,the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!
```



In [28]:
response = get_completion(prompt)

In [32]:
print(f'Original:\n{customer_email.strip()}\n\nTransformed:\n{response}\n\n')

Original:
Arrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse,the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!

Transformed:
Arrgh, I'm quite upset that my blender lid flew off and splattered my kitchen walls with smoothie! And to make matters worse, the warranty doesn't cover the cost of cleaning up my kitchen. I really need your help at this moment, my friend.




<a id='sect2'></a>
## <b><font color='darkblue'>Chat API : LangChain</font></b> ([back](#sect0))
* <b><font size='3ptx'><a href='#sect2_1'>Model</a></font></b>
* <b><font size='3ptx'><a href='#sect2_2'>Prompt template</a></font></b>
* <b><font size='3ptx'><a href='#sect2_3'>Output Parsers</a></font></b>

Let's try how we can do the same using [**LangChain**](https://github.com/langchain-ai/langchain).

In [36]:
#!pip install -U langchain-openai

<a id='sect2_1'></a>
### <b><font color='darkgreen'>Model</font></b>

In [37]:
from langchain_openai import ChatOpenAI

In [38]:
# To control the randomness and creativity of the generated
# text by an LLM, use temperature = 0.0
chat = ChatOpenAI(temperature=0.0, model=llm_model)
chat

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7f446c270d50>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7f446c172990>, temperature=0.0, openai_api_key=SecretStr('**********'), openai_proxy='')

<a id='sect2_2'></a>
### <b><font color='darkgreen'>Prompt template</font></b> ([back](#sect2))
<b><font size='3ptx'>Prompt templates are predefined recipes for generating prompts for language models. ([more](https://python.langchain.com/v0.1/docs/modules/model_io/prompts/quick_start/))</font></b>

A prompt for a language model is a set of instructions or input provided by a user to guide the model's response, helping it understand the context and generate relevant and coherent language-based output, such as answering questions, completing sentences, or engaging in a conversation ([more](https://python.langchain.com/v0.1/docs/modules/model_io/prompts/)).

In [40]:
from langchain.prompts import ChatPromptTemplate

In [39]:
template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
"""

The prompt to [**chat models**](https://python.langchain.com/v0.1/docs/modules/model_io/chat/) is a list of [chat messages](https://python.langchain.com/v0.1/docs/modules/model_io/chat/message_types/).

Each chat message is associated with content, and an additional parameter called <b><font color='darkblue'>role</font></b>. For example, in the OpenAI [Chat Completions API](https://platform.openai.com/docs/guides/chat/introduction), a chat message can be associated with an AI assistant, a human or a system role.

Create a chat prompt template like this:

In [41]:
prompt_template = ChatPromptTemplate.from_template(template_string)

In [45]:
# Show the original prompt template
print(prompt_template.messages[0].prompt.template)

Translate the text that is delimited by triple backticks into a style that is {style}. text: ```{text}```



In [46]:
# Show the original prompt input variable(s)
prompt_template.messages[0].prompt.input_variables

['style', 'text']

In [48]:
# Now let's define the content of two required variables
customer_style = """American English \
in a calm and respectful tone
"""

customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse, \
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

In [49]:
# Compose the final prompt:
customer_messages = prompt_template.format_messages(
                    style=customer_style,
                    text=customer_email)

In [50]:
print(type(customer_messages))
print(type(customer_messages[0]))

<class 'list'>
<class 'langchain_core.messages.human.HumanMessage'>


In [51]:
print(customer_messages[0])

content="Translate the text that is delimited by triple backticks into a style that is American English in a calm and respectful tone\n. text: ```\nArrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse, the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!\n```\n"


In [53]:
# Call the LLM to translate to the style of the customer message
customer_response = chat.invoke(customer_messages)

In [54]:
customer_response

AIMessage(content="I'm really frustrated that my blender lid flew off and splattered my kitchen walls with smoothie! And to make matters worse, the warranty doesn't cover the cost of cleaning up my kitchen. I could really use your help right now, friend.", response_metadata={'token_usage': {'completion_tokens': 50, 'prompt_tokens': 89, 'total_tokens': 139}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-959b9ab6-2fda-49e2-84d6-876f0240b33b-0')

In [55]:
print(customer_response.content)

I'm really frustrated that my blender lid flew off and splattered my kitchen walls with smoothie! And to make matters worse, the warranty doesn't cover the cost of cleaning up my kitchen. I could really use your help right now, friend.


In [56]:
service_reply = """Hey there customer, \
the warranty does not cover \
cleaning expenses for your kitchen \
because it's your fault that \
you misused your blender \
by forgetting to put the lid on before \
starting the blender. \
Tough luck! See ya!
"""

service_style_pirate = """\
a polite tone \
that speaks in English Pirate\
"""

In [57]:
service_messages = prompt_template.format_messages(
    style=service_style_pirate,
    text=service_reply)

print(service_messages[0].content)

Translate the text that is delimited by triple backticks into a style that is a polite tone that speaks in English Pirate. text: ```Hey there customer, the warranty does not cover cleaning expenses for your kitchen because it's your fault that you misused your blender by forgetting to put the lid on before starting the blender. Tough luck! See ya!
```



In [58]:
service_response = chat.invoke(service_messages)

In [59]:
print(service_response.content)

Ahoy there, valued customer! Regrettably, the warranty be not coverin' the cost o' cleanin' yer galley due to yer own negligence. Ye see, 'twas yer own doin' that ye forgot to secure the lid afore startin' the blender. 'Tis a tough break, indeed! Fare thee well!


<a id='sect2_3'></a>
### <b><font color='darkgreen'>Output Parsers</font></b> ([back](#sect2))
<b><font size='3ptx'>Output parsers are responsible for taking the output of an LLM and transforming it to a more suitable format.</font> This is very useful when you are using LLMs to generate any form of structured data. ([more](https://python.langchain.com/v0.1/docs/modules/model_io/output_parsers/))</b>

Language models output text. But many times you may want to get more structured information than just text back. This is where output parsers come in. 

Let's start with defining how we would like the LLM output to look like:
```json
{
  "gift": False,
  "delivery_days": 5,
  "price_value": "pretty affordable!"
}
```

Consider we have a customer review as below:

In [60]:
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

Then we define a review template to extract the information we are interested in:

In [63]:
review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""

In [71]:
prompt_template = ChatPromptTemplate.from_template(review_template)
print(f'Template:\n{prompt_template.messages[0].prompt.template}\n')
print(f'Input variable: {prompt_template.messages[0].prompt.input_variables}')

Template:
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}


Input variable: ['text']


In [73]:
messages = prompt_template.format_messages(text=customer_review)
response = chat.invoke(messages)

In [74]:
print(response.content)

{
  "gift": true,
  "delivery_days": 2,
  "price_value": ["It's slightly more expensive than the other leaf blowers out there"]
}


Even the output look like a json format, but it is a string:

In [75]:
type(response.content)

str

#### <b>Parse the LLM output string into a Python dictionary</b>

In [76]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

In [77]:
gift_schema = ResponseSchema(name="gift",
                             description="Was the item purchased\
                             as a gift for someone else? \
                             Answer True if yes,\
                             False if not or unknown.")
delivery_days_schema = ResponseSchema(name="delivery_days",
                                      description="How many days\
                                      did it take for the product\
                                      to arrive? If this \
                                      information is not found,\
                                      output -1.")
price_value_schema = ResponseSchema(name="price_value",
                                    description="Extract any\
                                    sentences about the value or \
                                    price, and output them as a \
                                    comma separated Python list.")

response_schemas = [gift_schema, 
                    delivery_days_schema,
                    price_value_schema]

In [78]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [79]:
format_instructions = output_parser.get_format_instructions()

In [81]:
type(format_instructions)

str

In [83]:
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"gift": string  // Was the item purchased                             as a gift for someone else?                              Answer True if yes,                             False if not or unknown.
	"delivery_days": string  // How many days                                      did it take for the product                                      to arrive? If this                                       information is not found,                                      output -1.
	"price_value": string  // Extract any                                    sentences about the value or                                     price, and output them as a                                     comma separated Python list.
}
```


In [84]:
review_template_2 = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product\
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

text: {text}

{format_instructions}
"""

In [85]:
prompt = ChatPromptTemplate.from_template(template=review_template_2)
messages = prompt.format_messages(
    text=customer_review, format_instructions=format_instructions)

In [86]:
print(messages[0].content)

For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the productto arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,and output them as a comma separated Python list.

text: This leaf blower is pretty amazing.  It has four settings:candle blower, gentle breeze, windy city, and tornado. It arrived in two days, just in time for my wife's anniversary present. I think my wife liked it so much she was speechless. So far I've been the only one using it, and I've been using it every other morning to clear the leaves on our lawn. It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features.


The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```

In [87]:
response = chat.invoke(messages)

In [88]:
print(response.content)

```json
{
	"gift": true,
	"delivery_days": 2,
	"price_value": ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]
}
```


In [89]:
output_dict = output_parser.parse(response.content)

In [90]:
type(output_dict)

dict

In [91]:
output_dict

{'gift': True,
 'delivery_days': 2,
 'price_value': ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]}

In [92]:
output_dict.get('delivery_days')

2

## <b><font color='darkblue'>Supplement</font></b>
* [Deeplearning.ai - Langchain Ch2: Model, prompt and parser](https://github.com/johnklee/ml_articles/blob/master/deeplearning_ai/langchain/ch2_model_prompt_and_parser.ipynb)
* [Deeplearning.ai - Langchain Ch3: Memory](https://github.com/johnklee/ml_articles/blob/master/deeplearning_ai/langchain/ch3_memory.ipynb)
* [Deeplearning.ai - Langchain Ch4: Chain](https://github.com/johnklee/ml_articles/blob/master/deeplearning_ai/langchain/ch4_chains.ipynb)
* [Deeplearning.ai - Langchain Ch5: Question and answer](https://github.com/johnklee/ml_articles/blob/master/deeplearning_ai/langchain/ch5_question_and_answer.ipynb)
* [Deeplearning.ai - Langchain Ch6: Evaluation](https://github.com/johnklee/ml_articles/blob/master/deeplearning_ai/langchain/ch6_evaluation.ipynb)
* [Deeplearning.ai - Langchain Ch7: Agents](https://github.com/johnklee/ml_articles/blob/master/deeplearning_ai/langchain/ch7_agents.ipynb)